In [1]:
# Import necessary libraries
%matplotlib inline
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, confusion_matrix, precision_score, recall_score, f1_score
from logging import StreamHandler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error

In [3]:
#cars = pd.read_csv("/content/cars_2010_2020.csv")
cars = pd.read_csv("cars_2010_2020.csv")

In [4]:
cars.head()

,Make,Model,Year,Engine Size (L),Fuel Type,Price (USD)
0,Volkswagen,Jetta,2010,4.2,Petrol,54073.09
1,Honda,Pilot,2017,4.2,Hybrid,44924.91
2,Nissan,Murano,2011,4.2,Hybrid,76963.44
3,Toyota,RAV4,2010,2.4,Petrol,30871.25
4,Nissan,Altima,2010,3.6,Petrol,72037.65


In [5]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Make             10000 non-null  object 
 1   Model            10000 non-null  object 
 2   Year             10000 non-null  int64  
 3   Engine Size (L)  10000 non-null  float64
 4   Fuel Type        10000 non-null  object 
 5   Price (USD)      10000 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 468.9+ KB


In [6]:
cars.describe()

,Year,Engine Size (L),Price (USD)
count,10000.000000,10000.000000,10000.000000
mean,2014.969900,2.992240,47397.584981
std,3.174303,1.150697,18643.431581
min,2010.000000,1.000000,15010.250000
25%,2012.000000,2.000000,31214.385000
50%,2015.000000,3.000000,47272.105000
75%,2018.000000,4.000000,63325.882500
max,2020.000000,5.000000,79998.870000


In [7]:
cars.isna().sum()

Make               0
Model              0
Year               0
Engine Size (L)    0
Fuel Type          0
Price (USD)        0
dtype: int64

In [9]:
cars.duplicated().sum()

0

In [10]:
plt.figure(figsize=(15 , 10))
sns.boxplot(cars)

<Axes: >

In [11]:
sns.pairplot(cars)

In [12]:
# Encoding non-numeric columns before correlation
# Encode non-numeric columns

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
cars['Make'] = encoder.fit_transform(cars['Make'])  # Encoding for 'Make'
cars['Model'] = encoder.fit_transform(cars['Model'])  # Encoding for 'Model'
cars['Fuel Type'] = encoder.fit_transform(cars['Fuel Type'])  # Encoding for 'Fuel Type'

# Compute correlations
correlation_matrix = cars.corr()
print(correlation_matrix)

                     Make     Model      Year  Engine Size (L)  Fuel Type  \
Make             1.000000  0.179059 -0.000230        -0.005727  -0.001840   
Model            0.179059  1.000000  0.010837        -0.018980   0.004416   
Year            -0.000230  0.010837  1.000000        -0.007676  -0.006079   
Engine Size (L) -0.005727 -0.018980 -0.007676         1.000000  -0.002237   
Fuel Type       -0.001840  0.004416 -0.006079        -0.002237   1.000000   
Price (USD)      0.007241 -0.025594 -0.004651         0.002370  -0.010987   

                 Price (USD)  
Make                0.007241  
Model              -0.025594  
Year               -0.004651  
Engine Size (L)     0.002370  
Fuel Type          -0.010987  
Price (USD)         1.000000  


In [13]:
sns.heatmap(cars.corr(), annot=True, cmap="coolwarm")

<Axes: >

In [14]:
# Feature Engineering

cars['Age'] = 2020 - cars['Year']

cars.head()

,Make,Model,Year,Engine Size (L),Fuel Type,Price (USD),Age
0,9,27,2010,4.2,3,54073.09,10
1,4,33,2017,4.2,2,44924.91,3
2,7,30,2011,4.2,2,76963.44,9
3,8,37,2010,2.4,3,30871.25,10
4,7,7,2010,3.6,3,72037.65,10


In [15]:
cars.corr()

,Make,Model,Year,Engine Size (L),Fuel Type,Price (USD),Age
Make,1.000000,0.179059,-0.000230,-0.005727,-0.001840,0.007241,0.000230
Model,0.179059,1.000000,0.010837,-0.018980,0.004416,-0.025594,-0.010837
Year,-0.000230,0.010837,1.000000,-0.007676,-0.006079,-0.004651,-1.000000
Engine Size (L),-0.005727,-0.018980,-0.007676,1.000000,-0.002237,0.002370,0.007676
Fuel Type,-0.001840,0.004416,-0.006079,-0.002237,1.000000,-0.010987,0.006079
Price (USD),0.007241,-0.025594,-0.004651,0.002370,-0.010987,1.000000,0.004651
Age,0.000230,-0.010837,-1.000000,0.007676,0.006079,0.004651,1.000000


In [16]:
sns.heatmap(cars.corr(), annot=True, cmap="coolwarm")

<Axes: >

In [17]:
from re import X
from sklearn.preprocessing import StandardScaler

# Select features and target variable

X = cars[['Year', 'Engine Size (L)', 'Age']]
y = cars['Price (USD)']

# Split the data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features

car_scaler = StandardScaler()
X_train_scaled = car_scaler.fit_transform(X_train)
X_test_scaled = car_scaler.transform(X_test)

In [18]:
# Initialize the model
lr = LinearRegression()  # Use LinearRegression
dtr = DecisionTreeRegressor()

# Train the model
lr.fit(X_train_scaled, y_train)

# Make predictions
y_pred = lr.predict(X_test_scaled)

# Evaluate the model using regression metrics
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")
print(f"Coefficients: {lr.coef_}")
print(f"Intercept: {lr.intercept_}")

Mean Squared Error: 344949268.7327494
R-squared: -0.00011759547986112118
Coefficients: [-3.62122952e+15  1.15500000e+02 -3.62122952e+15]
Intercept: 47380.70118507039


In [19]:
lr.predict(X_test_scaled)
pd.DataFrame({"Actual": y_test, "Predicted": lr.predict(X_test_scaled)})

,Actual,Predicted
6252,31048.59,47475.064305
4684,60267.40,47444.961830
1731,75805.42,47343.474205
4742,43526.75,47325.354843
4521,33572.46,47285.354843
...,...,...
6412,39895.24,47603.581193
8285,31817.53,47094.354843
7853,42907.30,47375.354843
1095,75865.96,47583.183668


In [20]:
# Train the model
dtr.fit(X_train_scaled, y_train)

# Make predictions
y_pred = dtr.predict(X_test_scaled)
from sklearn.metrics import mean_squared_error, r2_score

# Evaluate the model using regression metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 371841327.7767196
R-squared: -0.07808622410564059


In [21]:
# Train the model
rfreg = RandomForestRegressor()
rfreg.fit(X_train_scaled, y_train)

# Make predictions
y_pred = rfreg.predict(X_test_scaled)
from sklearn.metrics import mean_squared_error, r2_score

# Evaluate the model using regression metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 372645206.2863517
R-squared: -0.08041692347214369


In [22]:
from sklearn.model_selection import GridSearchCV

# Define hyperparameters for tuning
param_grid = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize model
dt_model = DecisionTreeRegressor(random_state=42)

# Perform Grid Search
grid_search = GridSearchCV(estimator=dt_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Best parameters and evaluation
print("Best Hyperparameters:", grid_search.best_params_)
print("Best MSE:", -grid_search.best_score_)

Best Hyperparameters: {'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2}
Best MSE: 349471765.2333957


In [23]:
# Evaluate perfomance

# 1. Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=20)

# 2. Train the model on the training data
best_model = DecisionTreeRegressor(max_depth=3, min_samples_leaf=1, min_samples_split=2)
best_model.fit(X_train, y_train)

# 3. Make predictions on the test set
y_pred = best_model.predict(X_test)

# 4. Evaluate performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Test Set Mean Squared Error (MSE): {mse}")
print(f"Test Set R-squared (R²): {r2}")

Test Set Mean Squared Error (MSE): 353092329.4957923
Test Set R-squared (R²): -0.006470953227539766


In [24]:
!pip install scikeras


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.2/750.2 kB 18.9 MB/s eta 0:00:00


In [25]:
import pickle

# Save the trained model
with open('decision_tree_model.pkl', 'wb') as file:
    pickle.dump(grid_search.best_estimator_, file)

print("Model saved to decision_tree_model.pkl")

Model saved to decision_tree_model.pkl


In [26]:
pickle.dump(car_scaler, open("car_scaler.pkl", "wb"))